Copyright 2019 Intel Corporation

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Introduction

In Lesson 3 of *Anomaly Detection: Linear Methods*, we learned about linear methods used for anomaly detection: linear regression, principal component analysis (PCA) and one-class support vector machines (SVMs).

Here will apply all three techniques to detect anomalies in synthetic and real-world data.

# Learning Outcomes

You should walk away from this Python tutorial with:
1. Practical experience with linear regression models
2. Practical experience with PCA
3. Practical experience with one-class SVMs 

# Imports

In [ ]:
import numpy as np
np.set_printoptions(suppress=True, precision=4)

import sys
import matplotlib.pyplot as plt
import sklearn.decomposition as decomp
import sklearn.linear_model as linear_model
import sklearn.datasets as sk_data
from sklearn.preprocessing import StandardScaler
import numpy.linalg as nla
import sklearn.svm as svm
import pandas as pd

# Python and library versions

In [ ]:
packages = [np]

msg = f"""
Python Version: {sys.version}

library .      version
-------        -------"""
print(msg)

for package in packages:
    print(f"{package.__name__:11}    {package.__version__:>7}")

# Section 1: Linear regression models

In this section, we are going to use synthetic datasets to detect  anomalies using a linear regression. We'll apply what we've learned to a real-world dataset in the exercises.

The key idea: deviation from regression model--that is, the difference between the actual value and the prediction value--is a measure of how anomalous a point is.

A reminder of how linear regression models is used for anomaly detection: 

1. Split data into train and test datasets
2. Use train data sets to (i) get the parameters of the regression model and (ii) determine the threshold for anomalies
3. Apply the results to the test data to detect anomalies


### Data

Here we will work with the data shown in the lecture: exam grade and hours studied.

In [ ]:
# Approximately linear data
exam_data1 = np.array([[1, 2, 3, 4, 5],
                    [57, 70, 76, 84, 91]]).T
print(exam_data1)

For each pair of data points, the first value is the hours studies and the second is the exam grade (out of 100).

In [ ]:
# One anomaly replaces a normal point
exam_data2 = np.array([[1, 2, 3, 4, 5],
                      [57, 70, 99, 84, 91]]).T
print(exam_data2)

The anomaly is (3, 99).

To simplify our work, we will introduce a helper function that plots a straight line given the slope, intercept, axes (to create the figure) and the style of the line (to plot multiple lines on the same graph).

In [ ]:
def plot_mb(m, b, ax, style):
    'plot a line y=mx+b on a matplotlib axis'
    xs = ax.get_xlim()
    ax.plot(xs, m*xs + b, style) #style is type of line

### Analysis 

Plot the two datasets together so we can easily compare them.

In [ ]:
fig, axes = plt.subplots(1,2,sharex=True)

# Modify axes so they look nice
axes[0].set_xlim([0, 6.01])
start_x, end_x = axes[0].get_xlim()
stepsize_x = 1
axes[0].xaxis.set_ticks(np.arange(start_x, end_x, stepsize_x))
axes[0].yaxis.set_ticks_position('both')
axes[0].set_ylim([45, 108])
axes[1].set_ylim(axes[0].get_ylim())
axes[1].yaxis.tick_right() # Display tick values on the right for clarity
axes[1].yaxis.set_ticks_position('both')

# Fit a straight straight line to the linear data
lr = linear_model.LinearRegression().fit(exam_data1[:,0:1],
                                         exam_data1[:,1])
axes[0].plot(*exam_data1.T, 'ro')
plot_mb(lr.coef_, lr.intercept_, axes[0], 'b-' )

# Fit a straight straight line to data with anomaly
lr2 = linear_model.LinearRegression().fit(exam_data2[:,0:1],
                                         exam_data2[:,1])
axes[1].plot(*exam_data2.T, 'ro')

# Plot two linear fits: from data with anomaly and from normal data
plot_mb(lr2.coef_, lr2.intercept_, axes[1], 'b-')
plot_mb(lr.coef_, lr.intercept_, axes[1], 'g--')

axes[0].set_title('No anomaly')
axes[1].set_title('With anomaly')
fig.text(0.5, 0.02, 'Hours studied', ha='center', va='center')
fig.text(0.05, 0.5, 'Grade', ha='center', va='center', rotation='vertical')
plt.show()

As we can see, all of the data on the left lies close to the regression model (blue line), while for the data on the right we can see that there is a point that is far from the line. This point (3, 99) is the anomaly.

**Note:** There are two lines in the panel on the right: the regression model for the data with the anomaly (solid blue line) and the one found from the normal data from the left panel (dashed green line). The difference between these two lines is due to the anomaly itself (all other points are unchanged). That is, anomalies affect the regression model.

It is because anomalies affect the regression model that we do the train/test split to ensure that the linear fit reflects only the normal data. Of course, such an approach presumes that we can have (or can create) a training set with only normal data.    

We will now treat 'exam_data1' (no anomaly) as the train dataset and 'exame_data2' (with anomaly) as the test dataset.

First, we fit our train dataset. We will use the _LinearRegression_ function from_sklearn_ as we did above. Recall from the anomaly score is the square of the residual for each point.

In [ ]:
ftrs, tgt = exam_data1[:,0:1], exam_data1[:,1]
lr_train = linear_model.LinearRegression().fit(ftrs, tgt)
print(f'Slope: {lr_train.coef_}')
print(f'Intercept: {lr_train.intercept_:.{3}}')
train_scores = (tgt - lr_train.predict(ftrs))**2 
print(train_scores)

Let's set the threshold for anomaly detection to be the just above the maximum score from the train dataset.

In [ ]:
margin = 0.01
threshold = max(train_scores) + margin
print(f'Threshold: {threshold:.{3}}')

Now let's calculate the anomaly scores for the test data.

In [ ]:
def do_linreg_anomaly_scores(train, test):
    ftrs, tgt = train[:,0:1], train[:,1]
    lr_train = linear_model.LinearRegression().fit(ftrs, tgt)
    anom_score = (test[:,1] - lr_train.predict(test[:,0:1]))**2
    return anom_score

In [ ]:
print(do_linreg_anomaly_scores(exam_data1, exam_data2))

We see that the middle point (index=2) exceeds the threshold. It is the anomaly we introduced into the data. As a check, we can compare the anomaly score above with those of the training data.

In [ ]:
print(train_scores)

As expected, the two sets of scores differ only for the anomalous point.

# Section 2: Principal component analysis (PCA)

In this section, we are going to use synthetic datasets to detect  anomalies using PCA. We'll apply what we've learned to a real-world dataset in the exercises.

How to calculate PCA-based anomaly scores:

  1. Preprocess the data (if needed)
  2. Compute the principal components (PCs) of the centered data
  3. Project our examples onto the PCs
  4. Calculate the distance between the original and the projected examples
  5. Use the distance to score the anomalies


### Data

We start with the data shown in the lecture: a simple 2D dataset of synthetic data: ($X_1, X_2$). To demonstrate how PCA works, we take $X_2 = X_1/2$ for all pairs except one. where we made a small change: (0.5, 0.26). This point is the anomaly.

In [ ]:
pca_example = np.array([[-3, -1.5], [-2.5, -1.25], [-1.5, -0.75], 
                        [-1, -0.5], [-0.5, -0.25], [0, 0], [0.5, 0.26], 
                        [1, 0.5],  [1.5, 0.75], [2.5, 1.25], [3, 1.5]])

What is the mean of the data?

In [ ]:
mean_pca_example = np.mean(pca_example, axis=0, keepdims=True)
mean_pca_example

What about the variance? We want to make sure that the variance in each direction is approximately equal (otherwise something trivial, like changing the unit of measurement from meters to millimeters, will drastically change the principal components).

In [ ]:
var_pca_example = np.var(pca_example, axis=0, keepdims=True)
var_pca_example

Since mean of this data is (practically) zero, we don't need to do mean subtraction. We should correct for the different variances, however. The typical approach is to divide each component by its standard deviation, enforcing the variance in each direction is 1.

In [ ]:
scaled_pca_example = pca_example/np.sqrt(var_pca_example)

# show the variances are equal
scaled_pca_example.var(axis=0)

Let's plot the original (i.e. _unscaled_) data

In [ ]:
fig, ax = plt.subplots()
ax.scatter(pca_example[:,0], pca_example[:,1])
ax.set_ylabel('$X_2$')
ax.set_xlabel('$X_1$')
ax.set_title('Original data')
plt.show()

Can you spot the anomaly?

To show what PCA can do, we will apply it to our dataset and take a sneak peek at the results (using PCA from *sklearn*). We expect the first principal component to lie along the $X_2 = X_1/2$ line because that is the direction along which the data varies the most. And given the way we constructed the dataset, there will be almost no variation along the second principal component except for the anomaly.

In [ ]:
pca = decomp.PCA(n_components=2)
pca.fit(scaled_pca_example)

In [ ]:
pca_example_trf = pca.transform(scaled_pca_example) 
fig2, ax2 = plt.subplots()
ax2.scatter(pca_example_trf[:,0], pca_example_trf[:,1])
ax2.set_ylabel('$PC2$')
ax2.set_xlabel('$PC1$')
ax2.set_title('Principal components')
plt.show()

The anomaly is evident.

### Analysis

Now we'll proceed more formally. We create a function to carry out PCA on 2D data following the five steps we outlined at the beginning of this section.

Instead of doing the centering and mean normalization ourselves, we will use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) built into scikit-learn.

In [ ]:
def get_1d_projected_vectors(obs, pca_object):
    # Note: The projection of vector a (data) along vector b (PC1)
    # is given by  [b / len(b)]* (len (a) cos (theta))
    # where theta is the angle between and b and the term in 
    # square parenthesis is a unit vector in the b direction
    #
    # Since cos (theta) = dot(a,b)/(len(a)len(b))
    # the projection can be written as
    # projs = b*[dot(a,b)/len(b)^2]
    #
    # The term in square parethenses is y_lengths
    # The projs is calculated adding back the mean
    # subtracted previously to center the data
    #
    # This is a very explicit way of handling the calculation.
    # See notes in "higher dimension" to see a way of generalizing
    # this to higher dimensions, while encapsulating the vector math.
    ssX = StandardScaler()
    centered_data = ssX.fit_transform(obs)
    pca_dirs = pca_object.components_
    
    y_lengths = centered_data.dot(pca_dirs.T) / pca_dirs.dot(pca_dirs.T)
    centered_projs = y_lengths*(pca_dirs)
    
    # Return the data to its original uncentered (and unscaled) positions
    return ssX.inverse_transform(centered_projs)

def do_pca_anomaly_scores(obs, pca_object):
    projected_vectors = get_1d_projected_vectors(obs, pca_object)    
    return nla.norm(obs-projs, axis=1)

In [ ]:
def do_1d_pca_anomaly_scores(obs):
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_ylabel('$X_2$')
    ax.set_xlabel('$X_1$')
    ax.set_title('Original data with PCA')
    
    # draw data
    ax.scatter(*obs.T, label='data')

    # Step 1: center and scale the data
    ssX = StandardScaler()
    centered_data = ssX.fit_transform(obs)
    mean = ssX.mean_
    
    #for completeness, show mean on plot
    ax.scatter(*mean.T, c='k', marker='^', label='mean') 

    # Step 2: compute prinicpal components
    # Here we focus on first PC  (greatest proportion of variance)
    pca = decomp.PCA(n_components=1)
    pca.fit_transform(centered_data)
    pca_dirs = pca.components_

    # draw principal components
    pca_endpoints = np.r_[-3.5*ssX.inverse_transform(pca_dirs),
                           3.5*ssX.inverse_transform(pca_dirs)]
    ax.plot(*pca_endpoints.T, 'y', label='PC1')

    # Step 3: Project our examples onto the PCs
    # 
    projs = get_1d_projected_vectors(obs, pca)
    ax.plot(*projs.T, 'r.', label='projected data')
    ax.legend(loc='best')
    
    # Step 4: Calculate distance between original and projected examples
    # Step 5: Use the distance to score the anomalies
    # The distance is the Euclidean norm and 
    # we use it as the anomaly score
    return nla.norm(obs-projs, axis=1)

In [ ]:
pca_example_scores = do_1d_pca_anomaly_scores(pca_example)

The plot shows that data (blue), the data projected onto PC1 (red points on yellow line) and the mean of the data used for centering (black triangle). By construction, PC1 passes through the mean, which is located at (0,0)--hard to see because there are a data point and a projected data point there too. 

Because the data lies very close to PC1, it is hard to see the anomaly. Therefore, we look at the anomaly scores. 

In [ ]:
print(pca_example_scores)

We see one score (0.0101) which is much larger than the others. To what point does it correspond? 

In [ ]:
print(pca_example[np.argmax(pca_example_scores)])

It is the anomaly in the data. PCA worked!

We repeat the analysis with noisier data: two Gaussian clusters seeded with two anomalies at (6.0, 6.0) and (0.0, 10.0).

In [ ]:
blobs_X, y = sk_data.make_blobs(centers=[[0,0], [10,10]])
figure, axes = plt.subplots(figsize=(6,6))
axes.scatter(*blobs_X.T, c=y)

spike_1 = np.array([[6.0,6.0]]) # Anomaly 1
spike_2 = np.array([[0.0,10]])  # Anomaly 2
axes.scatter(*spike_1.T, c='r')
axes.scatter(*spike_2.T, c='g')
axes.set_aspect('equal')
axes.set_ylabel('$X_2$')
axes.set_xlabel('$X_1$')
axes.set_title('Original cluster data with two anomalies')

# Combine the data so that the last two points are the anomalies
cluster_data = np.concatenate([blobs_X, spike_1, spike_2])


Carry out PCA


In [ ]:
cluster_data_scores = do_1d_pca_anomaly_scores(cluster_data)

With this dataset, the difference between the original data and the projected data is apparent--only some of the variation in the data can be explained by projecting it onto a line. (Note that the mean (black triangle) gets projected onto PC1 as well.)

Look at the anomaly scores.

In [ ]:
print(cluster_data_scores)
print(cluster_data_scores.shape)

The point with the highest scored is one of our seeded anomalies.

In [ ]:
print(max(cluster_data_scores))
print(cluster_data[np.argmax(cluster_data_scores)])

Because of the way we constructed our dataset, we know that the last two points are the anomalies. 

The one at (0.0, 10.0), which has index=101, is the one we found above. 

The other one (6.0, 6.0), with index=100, has an anomaly score of 0.0886, which is very low. This is because it lies very close to PC1.


This example shows the limitations of PCA for anomaly detection with clustered data. For such data, proximity methods should also be tried (see lesson 4). 

## Higher dimensions

In the function `do_pca_anomaly_scores` we called the helper function `get_projected_vectors` which found the projection of the vectors in our dataset onto the principal direction. By subtracting the projected vector from the original vector, we could find the length of the orthogonal components that we used to score. 

The approach used worked with one principal direction, and relied heavily on vector math. We can ask the (fitted) `pca` object to do the heavy lifting for us:
1. `reduced = pca.tranform(X)` will take our $N$-dimensional vectors, and project them onto the $d$-dimensional subspace. In our example, $N=2$ and $d=1$.
2. `projected = pca.inv_transform(reduced)` will embed the reduced vectors back into our $N$-dimensional subspace
3. `X - projected` are the components of the vectors that don't lie in our subspace.

In [ ]:
def get_projected_vectors(X, pca, ssX=None):
    if not ssX:
        ssX = StandardScaler().fit(X)
    centered_data = ssX.transform(X)
    reduced = pca.transform(centered_data)
    # To get back to the original space, we need to undo the PCA
    # as well as undo the scaling/centering step.
    return ssX.inverse_transform(pca.inverse_transform(reduced))

In [ ]:
# Show that it does the same thing as the previous version
def do_pca_anomaly_scores(obs, n_components=1):
    ssX = StandardScaler()
    centered_data = ssX.fit_transform(obs)
    pca = decomp.PCA(n_components=n_components)
    pca.fit(centered_data)
    
    projected_vectors = get_projected_vectors(obs, pca)
    return nla.norm(obs - projected_vectors, axis=1)

# check that the two answers agree
do_pca_anomaly_scores(pca_example) - pca_example_scores

Our higher dimensional analog returns the same result as doing the vector arithmetic manually!

# Section 3: One-class support vector machine (SVM)

In this section, we will use the cluster dataset we created in the previous section to detect anomalies using one-class SVM. We'll apply what we've learned to a real-world dataset in the exercises.

As we discussed in the lecture, one-class SVM tries to find the hyperplane with the largest separation between the normal and anomaly classes. Typically, the data is not used as is, but transformed using the "kernel trick" (see lecture). 

We will use the one-class SVM implemented in *sklearn*, which uses the radial basis function (RBF) by default. The input for the RBF kernel is the parameter *gamma* and we use its default value as well.
(A discussion of kernel is beyond the scope of this lesson. For more information, see
https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html#sklearn.svm.OneClassSVM)



One-class SVM typically returns anomalies (-1) or normal points (+1).  We, however, are interested in scoring our points and then ranking them by score.  Therefore, we will use the `decision_function` provided which returns the signed distance to separating hyperplane (negative distances are anomalies). To be consistent with out previous convention, where larger positive scores reflect more anomalous points, we need to take the negative of the `decision_function` .

In [ ]:
def do_svm_anomaly_scores(obs):
    
    oc_svm = svm.OneClassSVM(gamma='auto').fit(obs)
    scores = oc_svm.decision_function(obs).flatten()
    
    # Find the largest score and use it to normalize the scores
    max_score = np.max(np.abs(scores))
    
    # scores from oc_svm use "negative is anomaly"
    # To follow our previous convention
    # we multiply by -1 and divide by the maximum score to get scores
    # in the range [-1, 1] with positive values indicating anomalies
    return - scores / max_score

Apply the one-class SVM to the cluster dataset of the previous section.
Look at both the raw scores and the top five points (by score).

In [ ]:
print(do_svm_anomaly_scores(cluster_data))
print (do_svm_anomaly_scores(cluster_data).argsort()[-5:])

The two seeded anomalies (index=100 and 101) are the two highest ranked points. The performance is better than PCA, but the process of anomaly detection is less transparent.

## Exercises

### Data

For all of the exercises, we will use a real-world dataset: the `ionosphere` dataset from the UCI Machine Learning Database repository. 
This dataset consists of radar returns from the ionosphere and was originally used to classify the returns as good (suitable for further research) or bad. More information is available here: https://archive.ics.uci.edu/ml/datasets/ionosphere.

For the purposes of this notebook, we have adapted the dataset for anomaly detection. We removed two columns: one with the class labels (good or bad) and another which was a constant (0) for all data instances. 

We read the data in as a pandas dataframe and then place it in numpy array for compatibility with our existing functions

In [ ]:
ion_df = pd.read_csv('ionosphere_data.csv', header=None)
ion_data = np.array(ion_df.values)

Look at the data

In [ ]:
ion_data.shape

In [ ]:
print(ion_data)

### Exercise #1

This exercise refers to Section 1 (linear regression model)
 
For the ionosphere dataset, there isn't a natural dependent variable, so we have to choose one. Create a function `linreg_anomaly_scores` that returns the anomalies scores of a linear regression model where two inputs are provided: the data and the index of the feature that is the dependent variable. We will use this function in Exercise 4 to analyze the ionosphere data.



**Note 1:** Note that since we don't have separate training data, for simplicity you should score the anomalies on all of the data.

**Note 2:** You may wish to use the following helper function. For data with _n_ features, it returns an array of length _n_ with all entries `False` except for a single `True` values at a specified index (_idx_).

In [ ]:
def idx_to_boolean(n, idx):
    select = np.zeros(n,dtype=np.bool)
    select[idx] = True
    return select

In [ ]:
# YOUR CODE HERE

### Exercise #2

This exercise refers to Section 2 (PCA).

Create a function `pca_anomaly_scores` that  returns the anomalies scores of a PCA model where two inputs are provided: the data and the number of components. We will use this function in Exercise 4 to analyze the ionosphere data.

In [ ]:
# YOUR CODE HERE

### Exercise #3

This exercise refers to Section 3 (one-class SVM).

Create a function `svm_anomaly_scores` that returns the anomalies scores of a one-class SVM model where two sets inputs are provided: the data and a kernel. We will use this function in Exercise 4 to analyze the ionosphere data.

In [ ]:
# Choose the default kernel to be linear

# YOUR CODE HERE

### Exercise #4

We now have three `[method]_outlier_scores` functions.  Apply each of them to the ionosphere dataset.  What are the top five most anomalous examples using each technique?

In [ ]:
methods = [linreg_anomaly_scores,
           pca_anomaly_scores,
           svm_anomaly_scores]

# YOUR CODE HERE

Three different methods, three different answers. Therefore more exploration is needed, which we encourage the reader to pursue. For example, choosing different target feature for the linear regression; increase the number of components in PCA; or change the kernel for one-class SVM. 

The last suggestion can be implemented readily for the RBF kernel, since we used it in Section 3.


In [ ]:
print(sorted(do_svm_anomaly_scores(ion_data).argsort()[-5:]))

Same as for PCA with one component.

# Summary

In this assignment you should have learned: 

1. How to apply linear regression models
2. How to apply PCA
3. How to apply one-class SVMs

Congratulations! This concludes the lesson.